In [11]:
import requests
from datetime import datetime, timedelta
import pandas as pd

class TokenManager:
    def __init__(self, username='Fedebohl',password='Fedecapo_01'):
        self.token_info = None
        self.user_data={
                        'username':username,
                        'password':password
                        }
        self.load_user_data()
        self.token_url = 'https://api.invertironline.com/token'
        self.base_url = 'https://api.invertironline.com/api/v2'
        self.portfolio_url = f'{self.base_url}/portafolio'
        self.quotes_url = f'{self.base_url}/Cotizaciones/{{instrument}}/{{country}}/Todos'
        self.his_url = f'{self.base_url}/bCBA/Titulos/{{ticker}}/Cotizacion/seriehistorica/2023-01-01/{{date}}/sinAjustar'
        self.get_new_token()

    def load_user_data(self):
        self.username = self.user_data['username']
        self.password = self.user_data['password']

    def get_new_token(self):
        data = {
            'grant_type': 'password',
            'username': self.username,
            'password': self.password
        }
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        response = requests.post(self.token_url, data=data, headers=headers)
        self.token_info = response.json()
        if 'error' in self.token_info:
            raise Exception(f"Error obtaining token: {self.token_info['error']}")
        self.token_info['expires_at'] = datetime.now() + timedelta(seconds=self.token_info['expires_in'])
    
    def refresh_token(self):
        data = {
            'grant_type': 'refresh_token',
            'refresh_token': self.token_info['refresh_token']
        }
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        response = requests.post(self.token_url, data=data, headers=headers)
        new_token_info = response.json()
        if 'error' in new_token_info:
            raise Exception(f"Error refreshing token: {new_token_info['error']}")
        self.token_info.update(new_token_info)
        self.token_info['expires_at'] = datetime.now() + timedelta(seconds=self.token_info['expires_in'])

    def ensure_token(self):
        if (not self.token_info) or (datetime.now() >= self.token_info['expires_at']):
            self.refresh_token() if self.token_info else self.get_new_token()

    def get_portfolio(self):
        self.ensure_token()
        headers = {'Authorization': f"Bearer {self.token_info['access_token']}"}
        response = requests.get(self.portfolio_url, headers=headers)
        if response.status_code != 200:
            raise Exception(f"Error fetching portfolio: {response.text}")
        port_df=pd.DataFrame(response.json()['activos'])
        tickers=[equity['simbolo'] for equity in port_df['titulo'].to_list()]
        tipos=[equity['tipo'] for equity in port_df['titulo'].to_list()]
        port_df=port_df.drop(columns=['cantidad','comprometido','puntosVariacion','ultimoPrecio','ppc','gananciaPorcentaje','gananciaDinero','parking','titulo'])
        port_df['simbolo']=tickers
        port_df['tipo']=tipos
        port_df['tipo']=port_df['tipo'].replace('TitulosPublicos', 'Bonos')
        port_df['tipo']=port_df['tipo'].replace('FondoComundeInversion','FCI')
        port_df['valorizado%']=round(100*port_df['valorizado']/sum(port_df['valorizado']),2)
        port_df['gananciaDiariaPonderada'] = port_df['variacionDiaria'] * port_df['valorizado%']/100
        return port_df

    def get_quotes(self, instrument):
        self.ensure_token()
        url = self.quotes_url.format(instrument=instrument, country=('argentina' if instrument!='aDRs' else "estados_Unidos"))
        headers = {'Authorization': f"Bearer {self.token_info['access_token']}"}
        response = requests.get(url=url,headers=headers)
        if response.status_code != 200:
            try:
                self.refresh_token()
                response = requests.get(url=url,headers=headers)
                if response.status_code != 200:
                    raise Exception(f"Error fetching {instrument} quotes: {response.text}")
            except: raise Exception(f"Error fetching {instrument} quotes: {response.text}")
        df=pd.DataFrame(response.json()['titulos'])
        df=df[['simbolo','ultimoPrecio','variacionPorcentual']]
        df=df.rename(columns={'variacionPorcentual':'Var%'})
        return df
    def get_his(self,ticker):
        self.ensure_token()
        url = self.his_url.format(ticker=ticker,date='2024-08-30')#datetime.today().strftime('%Y-%m-%d'))
        headers = {'Authorization': f"Bearer {self.token_info['access_token']}"}
        response = requests.get(url=url,headers=headers)
        if response.status_code != 200:
            try:
                self.refresh_token()
                response = requests.get(url=url,headers=headers)
                if response.status_code != 200:
                    raise Exception(f"Error fetching {ticker} quotes: {response.text}")
            except: raise Exception(f"Error fetching {ticker} quotes: {response.text}")
        df=pd.DataFrame(response.json())
        df=df[['ultimoPrecio','fechaHora']]
        return df  
    
    def get_operaciones_hist(self):
        self.ensure_token()
        operaciones_url = f"{self.base_url}/operaciones?filtro.estado=todas&filtro.fechaDesde=2020-01-01&filtro.fechaHasta={datetime.today().strftime('%Y-%m-%d')}&filtro.pais=argentina"
        headers = {'Authorization': f"Bearer {self.token_info['access_token']}"}
        response = requests.get(operaciones_url, headers=headers)
        if response.status_code != 200:
            raise Exception(f"Error fetching portfolio: {response.text}")
        df=pd.DataFrame(response.json())
        df['fechaOperada'] = pd.to_datetime(df['fechaOperada'].str.split('T').str[0], format='%Y-%m-%d', errors='coerce')
        df['fechaOrden'] = pd.to_datetime(df['fechaOrden'].str.split('T').str[0], format='%Y-%m-%d', errors='coerce')
        #df['fechaOperada']=df['fechaOperada'].dt.strftime('%Y-%m-%d')
        #df = df[df['fechaOperada'].notna()]
        #Ajuste por los BOPREALES
        filtro = (df['tipo'] == 'Pago de Amortización')
        cantidad_vendida=0
        filtered_df = df[(df['simbolo'] == 'BPO27') & (df['fechaOperada'] < pd.Timestamp('2024-03-01'))]
        for index, row in filtered_df.iterrows():
            if row['tipo'] == 'Compra':
                cantidad_vendida += row['cantidadOperada']
            elif row['tipo'] == 'Venta':
                cantidad_vendida -= row['cantidadOperada']
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'cantidadOperada'] = cantidad_vendida
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'precioOperado'] = 71000
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'montoOperado'] = 71000*cantidad_vendida
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'fechaOperada'] = pd.Timestamp('2024-03-01')
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'tipo'] = 'Venta'
        precios = {
            'BPOA7': 85000,
            'BPOB7': 75000,
            'BPOC7': 65000,
            'BPOD7': 58000
        }
        # Actualizar las filas de los nuevos bonos a "Compra" con sus precios y montos
        for simbolo, precio in precios.items():
            filtro_bono = filtro & (df['simbolo'] == simbolo)
            df.loc[filtro_bono, 'precioOperado'] = precio
            df.loc[filtro_bono, 'montoOperado'] = precio * df.loc[filtro_bono, 'cantidadOperada']
            df.loc[filtro_bono, 'fechaOperada'] = pd.Timestamp('2024-03-01')
            df.loc[filtro_bono, 'tipo'] = 'Compra'

        df=df[df['tipo'].isin(['Compra', 'Venta'])]
        df=df[df['estado']=='terminada']
        df=df[['tipo','fechaOperada','simbolo','cantidadOperada','montoOperado','precioOperado']]
        df=df.sort_values(by='fechaOperada', ascending=True)
        return df
    
    def get_operaciones(self,acciones_now,cedears_now,titpub):
        self.ensure_token()
        operaciones_url = f"{self.base_url}/operaciones?filtro.estado=todas&filtro.fechaDesde=2020-01-01&filtro.fechaHasta={datetime.today().strftime('%Y-%m-%d')}&filtro.pais=argentina"
        headers = {'Authorization': f"Bearer {self.token_info['access_token']}"}
        response = requests.get(operaciones_url, headers=headers)
        if response.status_code != 200:
            raise Exception(f"Error fetching portfolio: {response.text}")
        df=pd.DataFrame(response.json())
        df['fechaOperada'] = pd.to_datetime(df['fechaOperada'].str.split('T').str[0], format='%Y-%m-%d', errors='coerce')
        df['fechaOrden'] = pd.to_datetime(df['fechaOrden'].str.split('T').str[0], format='%Y-%m-%d', errors='coerce')
        #df['fechaOperada']=df['fechaOperada'].dt.strftime('%Y-%m-%d')
        #df = df[df['fechaOperada'].notna()]
        #Ajuste por los BOPREALES
        filtro = (df['tipo'] == 'Pago de Amortización')
        cantidad_vendida=0
        filtered_df = df[(df['simbolo'] == 'BPO27') & (df['fechaOperada'] < pd.Timestamp('2024-03-01'))]
        for index, row in filtered_df.iterrows():
            if row['tipo'] == 'Compra':
                cantidad_vendida += row['cantidadOperada']
            elif row['tipo'] == 'Venta':
                cantidad_vendida -= row['cantidadOperada']
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'cantidadOperada'] = cantidad_vendida
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'precioOperado'] = 71000
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'montoOperado'] = 71000*cantidad_vendida
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'fechaOperada'] = pd.Timestamp('2024-03-01')
        df.loc[(filtro & (df['simbolo']=='BPO27')), 'tipo'] = 'Venta'
        precios = {
            'BPOA7': 85000,
            'BPOB7': 75000,
            'BPOC7': 65000,
            'BPOD7': 58000
        }
        # Actualizar las filas de los nuevos bonos a "Compra" con sus precios y montos
        for simbolo, precio in precios.items():
            filtro_bono = filtro & (df['simbolo'] == simbolo)
            df.loc[filtro_bono, 'precioOperado'] = precio
            df.loc[filtro_bono, 'montoOperado'] = precio * df.loc[filtro_bono, 'cantidadOperada']
            df.loc[filtro_bono, 'fechaOperada'] = pd.Timestamp('2024-03-01')
            df.loc[filtro_bono, 'tipo'] = 'Compra'

        df=df[df['tipo'].isin(['Compra', 'Venta'])]
        df=df[df['estado']=='terminada']
        df=df[['tipo','fechaOperada','simbolo','cantidadOperada','montoOperado','precioOperado']]
        df=df.sort_values(by='fechaOperada', ascending=True)
        kind=[]
        for i in df.values.tolist():
            _='Accion' if i[2] in acciones_now['simbolo'].to_list() else ('Cedear' if i[2] in cedears_now['simbolo'].to_list() else ('Bono' if i[2] in titpub['simbolo'].to_list() else None))
            kind.append(_)
        df['Tipo de Acción']=kind
        df.columns=['Tipo Transacción','Fecha Liquidación','Simbolo','Cantidad','Monto','Precio Ponderado','Tipo de Acción']
        return df 

In [302]:
IOL=TokenManager()


## Pasos a realizar
1. Obtener todas las operaciones hasta la fecha
2. Sacar dividendos, amortizaciones y extras
3. Agrupar por ticker y luego por mes-->Va a ser rendimiento mensual del portafolio.
4. Crear un DataFrame donde las filas sean los meses hasta la fecha y las columnas la cantidad de cada acción/bono neto al final del mes
5. Buscar los precios en USD de cada activo
6. Crear un nuevo DataFrame con el valor en USD de cada acción al final del mes
7. Crear una columna de suma
8. Crear una columna de variación porcentual.
9. Anualizar a TNA el rendimiento.

- Un posible paso extra sería la posibilidad de analizar que activos rindieron más y en que fecha. Es decir explicar el rendimiento de la cartera.



 


In [ ]:
op_hist=IOL.get_operaciones_hist()
op_hist=op_hist.copy()
ratios={'JPM':3,
        'AAPL':2,
        'MELI':2,
        'VIST':3}
fecha_limite = pd.Timestamp('2024-01-26')
for ticker in ratios.keys():
    filtro = (op_hist['simbolo'] == ticker) & (op_hist['fechaOperada'] < fecha_limite)
    op_hist.loc[filtro, 'cantidadOperada'] *= ratios[ticker]
op_hist_copy=op_hist.copy()

op_hist

In [ ]:
import yfinance as yf
tickers={
    'YPFD':'YPF',
    'PAMP':'PAM',
    "BRKB":'BRK-B',
    'TGSU2':'TGS',
    'TECO2':'TEO',
    'AE38':'AE38D',
    'AL29':'AL29D',
    'AL30':'AL30D',
    'AL35':'AL35D',
    'AL41':'AL41D',
    'CO26':'CO26D',
    'GD29':'GD29D',
    'GD30':'GD30D',
    'GD35':'GD35D',
    'GD38':'GD38D',
    'GD41':'GD41D',
    'GD46':'GD46D',
    'T2X5':'T2X5D',
    'T4X4':'T4X4D',
    'TX26':'TX26D',
    'TX28':'TX28D',
    'BPOA7':'BPA7D',
    'BPOB7':'BPB7D',
    'BPOC7':'BPC7D',
    'BPOD7':'BPD7D',
    'BPJ25':'BPJ5D',
    'BPY26':'BPY6D'
}

tickers_usd={}
vars_usd={}
fails=[]
for ticker in op_hist['simbolo'].unique():
    try:
        data = yf.download(ticker if ticker not in tickers.keys() else tickers[ticker], start="2023-01-01", end=pd.Timestamp.today().strftime('%Y-%m-%d'), interval="1d")
        precios_mensuales = data['Adj Close'].resample('M').last()
        tickers_usd[ticker]=precios_mensuales
        open_=data['Open'].resample('M').first()
        close_=data['Close'].resample('M').last()
        vars_usd[ticker]=close_/open_-1
    except:
        fails.append(ticker)
        continue
fails

In [ ]:
op_hist=op_hist_copy.copy()
op_hist['fechaOperada']=[x.strftime('%Y-%m')for x in op_hist['fechaOperada']]
op_hist['cantidadOperada']=[op_hist.loc[x]['cantidadOperada'] if op_hist.loc[x]['tipo']=='Compra' else -op_hist.loc[x]['cantidadOperada'] for x in op_hist.index]
op_hist=op_hist.groupby(by=['fechaOperada','simbolo'])[['cantidadOperada']].sum()
op_hist

In [ ]:
df = pd.DataFrame(index=pd.date_range(start="2023-01-01", end=pd.Timestamp.today(), freq='M'), columns=[s for s in op_hist_copy['simbolo'].unique()])
df.index=[x.strftime('%Y-%m') for x in df.index]
df=df.loc[op_hist.index[0][0]:]
for ind in op_hist.index:
    df.at[ind[0],ind[1]]=op_hist.loc[ind][['cantidadOperada']].sum()
df

In [307]:
import math

df_acum={}
for col in df.columns:
    acumulado = 0
    valores_acumulados=[]
    # Iterar sobre la lista
    for valor in df[col]:
        if not math.isnan(valor):
            acumulado += valor
        # Agregar el valor acumulado actual a la lista
        valores_acumulados.append(acumulado)
    df_acum[col]=valores_acumulados
df_acum=pd.DataFrame(df_acum,index=df.index)

In [ ]:
import requests
from bs4 import BeautifulSoup

response=requests.get('https://marcosemmi.com/ratios-de-cedears/')
soup=BeautifulSoup(response.text,'html.parser')
table=soup.find('table')
rows = []
for tr in table.find_all('tr'):
    cells = tr.find_all('td')
    row = [cell.get_text(strip=True) for cell in cells]
    if row:  # Ignorar filas vacías
        rows.append(row)
ratios=pd.DataFrame(rows,columns=['ticker','nombre','cod bolsa','ratio','ticker-usd','area'])
ratios['ratio']=[float(x.split(':')[0]) for x in ratios['ratio']]
ratios.set_index('ticker',inplace=True)
ratios=ratios['ratio']
ratios

In [ ]:
cantXaccion = {
    'PAMP': 25,
    'YPFD': 1,
    'GGAL': 10,
    'CEPU': 10,
    'COME': 1,
    'TGSU2': 5,
    'CRES': 10,
    'EDN': 20,
    'LOMA': 5,
    'BMA': 10,
    'BBAR': 3,
    'SUPV': 5,
    'IRSA': 10,
    'TECO2': 5,
    'TS': 2,
    'IRCP': 4,
    'SPY': 20,
    'QQQ':20,
    'DIA':20,
    'IWM':10
}


price_usd=pd.DataFrame(tickers_usd)
price_usd.index=[x.strftime('%Y-%m') for x in price_usd.index]
price_usd=price_usd.loc[op_hist.index[0][0]:]
for col in price_usd.columns:
    if col in cantXaccion.keys():
        price_usd[col]=price_usd[col]/cantXaccion[col]
    elif col in ratios.index:
        price_usd[col]=price_usd[col]/ratios[col]
price_usd

In [ ]:
df_val=df_acum.copy()[price_usd.columns]  #CUANDO CONSIGA DATOS DE BONOS DEBERÍA IR "df.columns"
for col in df_val.columns:
    for ind in df_val.index:
        df_val.at[ind,col]=df_acum.at[ind,col]*price_usd.at[ind,col]
df_val

In [ ]:
df_val['Portfolio']=[sum(df_val.loc[x]) for x in df_val.index]
df_val

In [ ]:
vars_usd=pd.DataFrame(vars_usd)
vars_usd.index=[x.strftime('%Y-%m') for x in vars_usd.index]
vars_usd=vars_usd.loc[op_hist.index[0][0]:]
spy=vars_usd['SPY']
vars_usd

In [ ]:
var_pond=df_val.copy()
for col in var_pond.columns:
    var_pond[col]=var_pond[col]/var_pond['Portfolio']
var_pond=var_pond.drop(columns=['Portfolio'])
var_pond

In [ ]:
for ind in var_pond.index:
    for col in var_pond.columns:
        var_pond.at[ind,col]*=vars_usd.at[ind,col]
var_pond

In [ ]:
var_pond['Portfolio']=[sum(var_pond.loc[x]) for x in var_pond.index]
var_pond

In [319]:
from plotly import graph_objects as go

fig=go.Figure()
fig.add_trace(go.Scatter(x=var_pond.index,y=var_pond['Portfolio'],name='Portfolio'))
fig.add_trace(go.Scatter(x=spy.index,y=spy,name='SPY'))
fig.show()


In [ ]:
print(var_pond['Portfolio'].mean())
print(spy.mean())